In [44]:
from databases import PosicaoDm1,Crm,BDS
from funcoes_datas import FuncoesDatas
import pandas as pd
from datetime import datetime,date,timedelta
from dateutil.relativedelta import relativedelta

In [45]:
#Chamando as classesdo DataBases
posicao = PosicaoDm1()
crm= Crm()
fdt = FuncoesDatas()
bds = BDS()
fdt = FuncoesDatas()

#Definir os parametros de data do pickle

data = bds.banco.hoje()
data = data + relativedelta(months=-6)
data = data - relativedelta(days=data.day -1)
data = fdt.workday(data=data, n_dias=-1)

hoje = fdt.hoje()

#Trazer todos os produtos e suas indormações

produtos = crm.rating_produtos()
produtos['GuidProduto']=produtos['GuidProduto'].str.lower()

#PUxar a posição 6 meses para trás
#definir nome do arquivo pickle

arquivo = str(data.year)+'-'+str(data.month)+'-'+str(data.day)

try:
    arquivo_fundos_ant = pd.read_pickle ('O:/SAO/CICH_All/Portfolio Management/Arquivos/Pickle/PosicaoDm1/df_fundos_' + arquivo)
    arquivo_pf_ant = pd.read_pickle ('O:/SAO/CICH_All/Portfolio Management/Arquivos/Pickle/PosicaoDm1/df_tit_' + arquivo)
    
except:
    arquivo_fundos_ant = pd.read_pickle ('O:/SAO/CICH_All/Portfolio Management/Arquivos/Pickle/PosicaoDm1/df_fundos_2023-12-26.pickle')
    arquivo_pf_ant = pd.read_pickle ('O:/SAO/CICH_All/Portfolio Management/Arquivos/Pickle/PosicaoDm1/df_tit_2023-12-26.pickle')

carteira_ant = pd.concat([arquivo_fundos_ant,arquivo_pf_ant]) 
carteira_ant['GuidProduto']= carteira_ant['GuidProduto'].str.lower()


#Posicao em d1
carteira_atual = posicao.posicao_total_depara(hoje)[['DataArquivo','NomeContaCRM','TipoProduto','GuidProduto','NomeProduto','QuantidadeFinal','FinanceiroFuturo']]
carteira_atual=carteira_atual.sort_values(by=['NomeContaCRM','QuantidadeFinal','NomeProduto'])
carteira_atual['GuidProduto']= carteira_atual['GuidProduto'].str.lower()

In [48]:
#verificar os produtos novos
produtos_ant = carteira_ant[['GuidProduto','NomeProduto']].drop_duplicates()
produtos_atual = carteira_atual[['GuidProduto','NomeProduto']].drop_duplicates()

#Verificar produtos novos
verificar = pd.merge(left= produtos_atual,right=produtos_ant ,on='GuidProduto',how='left')
# verificar = verificar[verificar['NomeProduto_y'].isnull()]
# verificar = pd.merge(left = verificar,right=produtos, on='GuidProduto',how='left')

# #retirar caixa e compromissada da analise
# analise = pd.merge(left = carteira_atual,right=verificar,on='GuidProduto',how='right')
# analise = analise[(analise['TipoProduto']!='CAIXA')&(analise['TipoProduto']!='COMP')].drop(columns=['NomeProduto_x','NomeProduto_y','Ativo'])
# analise['ISIN']=analise['ISIN'].str.lower()
# analise['CETIP']=analise['CETIP'].str.lower()
# analise = analise[(analise['Rating']!='Recomendado')].drop_duplicates()



In [49]:
verificar.to_excel('ver.xlsx')

In [41]:
#Retorna os produtos aprovados no comite
produtos_comite = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Consolidado - Ativos Analisados v3.xlsx')
produtos_comite = produtos_comite[produtos_comite['Deliberação'].str.contains('Zerar na origem')]
produtos_comite['Cetip/Isin/CNPJ']=produtos_comite['Cetip/Isin/CNPJ'].str.lower()
produtos_comite = produtos_comite.dropna(subset=['Cetip/Isin/CNPJ'])
produtos_comite = produtos_comite[produtos_comite['On/Off']!='OFF']

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
<ipython-input-41-c025ad703322>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  produtos_comite['Cetip/Isin/CNPJ']=produtos_comite['Cetip/Isin/CNPJ'].str.lower()


In [39]:
#Filtrando tudo o que foi adquirido sem comite
cnpj = pd.merge(left = produtos_comite,right=analise,left_on='Cetip/Isin/CNPJ',right_on='CNPJ',how='right')
isin =  pd.merge(left = produtos_comite,right=analise,left_on='Cetip/Isin/CNPJ',right_on='ISIN',how='right')
cetip = pd.merge(left = produtos_comite,right=analise,left_on='Cetip/Isin/CNPJ',right_on='ISIN',how='right')
ativos_vender = pd.concat([cnpj,isin,cetip])
ativos_vender=ativos_vender[ativos_vender['Cliente'].isnull()]
ativos_vender = ativos_vender[['Data Comitê','Cliente','NomeContaCRM',
       'NomeProduto','Deliberação','Observação', 'CAP', 'QuantidadeFinal', 'FinanceiroFuturo','ISIN',
       'CETIP', 'CNPJ', 'emissor', 'comite', 'vencimento', 'Rating']].drop_duplicates()
ativos_vender['Comitê']='Sem_aprovação'
ativos_vender = ativos_vender[ativos_vender['NomeContaCRM'].str.contains("Monitorado")== False].drop_duplicates()
ativos_vender.to_excel('verificar.xlsx')


In [40]:
#puzando a carteira recente
carteira = pd.merge(left= carteira_atual,right=produtos ,on='GuidProduto',how='left')
carteira = carteira[(carteira['TipoProduto']!='CAIXA')&(carteira['TipoProduto']!='COMP')]
carteira['ISIN']=carteira['ISIN'].str.lower()
carteira['CETIP']=carteira['CETIP'].str.lower()

#cruzando os dados para encontrar o ativo na carteira
cnpj = pd.merge(left = produtos_comite,right=carteira,left_on='Cetip/Isin/CNPJ',right_on='CNPJ',how='left')
isin =  pd.merge(left = produtos_comite,right=carteira,left_on='Cetip/Isin/CNPJ',right_on='ISIN',how='left')
cetip = pd.merge(left = produtos_comite,right=carteira,left_on='Cetip/Isin/CNPJ',right_on='CETIP',how='left')
ativos_vender_comite = pd.concat([cnpj,isin,cetip])
ativos_vender_comite=ativos_vender_comite[ativos_vender_comite['Ativo_y'].notnull()]
ativos_vender_comite = ativos_vender_comite[['Data Comitê','Cliente','NomeContaCRM',
       'NomeProduto','Deliberação','Observação', 'CAP', 'QuantidadeFinal', 'FinanceiroFuturo','ISIN',
       'CETIP', 'CNPJ', 'emissor', 'comite', 'vencimento', 'Rating']].drop_duplicates()
ativos_vender_comite['Comitê']='Aprovado comitê'

In [13]:
ativos_n_aprovados = pd.concat([ativos_vender,ativos_vender_comite])

In [14]:
ativos_n_aprovados.to_excel('verificar.xlsx')